# 🚀 Master Python - Corso Intensivo
## Sessione 1 - Parte 2: Tre Progetti Completi

---

Questo notebook ti guiderà attraverso l'apprendimento di Python avanzato con 3 progetti completi e analisi dettagliate di ogni concetto.

**Iniziamo!**

In [ ]:
# Esegui il file con tutti i progetti
%run "C:/Users/Marco/Desktop/MasterPython/session1_part2_projects.py"

# 📊 AN ALISI DETTAGLIATA DEI TRE PROGETTI

## 🎯 Metodologia di Problem Solving

Prima di immergerci nel codice, capiamo come approcciarsi a progetti complessi:

### 1. **Comprendere il Problema**
```
❓ Domande chiave:
- Qual è l'obiettivo finale?
- Quali sono gli input e output?
- Quali vincoli devo rispettare?
- Ci sono casi edge da considerare?
```

### 2. **Scomporre in Sottoproblemi**
```
📦 Strategia Divide-et-Impera:
Problema Grande → Problemi Piccoli → Soluzione Incrementale
```

### 3. **Scegliere Strutture Dati e Pattern**
```
🧰 Toolbox Python:
- Dataclasses per dati strutturati
- Enum per costanti
- Context Managers per risorse
- Protocols per interfacce
```

### 4. **Implementare con Best Practices**
```
✅ Clean Code:
- Type hints ovunque
- Logging per debug
- Error handling robusto
- Testing
```

# 📈 PROGETTO 1: Trading System v4.0 - Analisi Approfondita

## 🔍 Panoramica del Problema

**Obiettivo**: Creare un sistema di trading automatico che:
1. Analizza mercati crypto in tempo reale (simulato)
2. Esegue trade basati su indicatori tecnici
3. Salva tutto in un database SQLite
4. Genera report statistici

## 🧠 Concetti Chiave Spiegati

### 1️⃣ **Protocol Pattern** - Interfacce Type-Safe

```python
class TradingStrategy(Protocol):
    def analyze(self, data: Dict) -> float:
        ...
```

**Perché usarlo?**
- Definisce un "contratto" che le classi devono rispettare
- Type checking statico (mypy può verificare)
- Flessibilità: puoi sostituire strategie senza modificare il bot

**Esempio pratico:**
```python
# Puoi creare diverse strategie che rispettano il Protocol
class RSIStrategy:
    def analyze(self, data: Dict) -> float:
        if data['rsi'] < 30:
            return 1.0  # Strong BUY
        elif data['rsi'] > 70:
            return -1.0  # Strong SELL
        return 0.0

class MomentumStrategy:
    def analyze(self, data: Dict) -> float:
        if data['change_24h'] > 5:
            return 1.0
        elif data['change_24h'] < -5:
            return -1.0
        return 0.0

# Entrambe funzionano con TradingBot!
```

### 2️⃣ **Dataclasses con Validazione** - Meno Boilerplate, Più Sicurezza

```python
@dataclass
class Trade:
    symbol: str
    quantity: float
    price: float
    timestamp: datetime = field(default_factory=datetime.now)
    
    def __post_init__(self):
        if self.quantity <= 0:
            raise ValueError("Quantity must be positive")
```

**Cosa fa `@dataclass`?**
- Auto-genera `__init__`, `__repr__`, `__eq__`, `__hash__`
- Riduce il codice da scrivere del 70%!

**Prima (senza dataclass):**
```python
class Trade:
    def __init__(self, symbol, quantity, price, timestamp=None):
        self.symbol = symbol
        self.quantity = quantity
        self.price = price
        self.timestamp = timestamp or datetime.now()
        if self.quantity <= 0:
            raise ValueError("Quantity must be positive")
    
    def __repr__(self):
        return f"Trade(symbol={self.symbol}, quantity={self.quantity}, ...)"
    
    def __eq__(self, other):
        return (self.symbol == other.symbol and 
                self.quantity == other.quantity and ...)
```

**Dopo (con dataclass):**
```python
@dataclass
class Trade:
    symbol: str
    quantity: float
    price: float
    timestamp: datetime = field(default_factory=datetime.now)
    
    def __post_init__(self):
        if self.quantity <= 0:
            raise ValueError("Quantity must be positive")
```

**Risparmio: 15 righe → 7 righe!**

### 3️⃣ **Context Manager** - Gestione Automatica delle Risorse

```python
def __enter__(self):
    logger.info("Trading bot started")
    return self

def __exit__(self, exc_type, exc_val, exc_tb):
    # Cleanup SEMPRE eseguito
    self.calculate_indicators.cache_clear()
    if os.path.exists(self.db_path):
        os.remove(self.db_path)
    return False
```

**Uso:**
```python
with TradingBot() as bot:
    bot.run_simulation()
# Cleanup automatico qui! Anche se c'è un errore!
```

**Perché è fondamentale?**
```python
# ❌ MALE (rischio memory leak!)
bot = TradingBot()
bot.run_simulation()
# Se c'è un errore, il cleanup non viene eseguito!

# ✅ BENE (cleanup garantito)
with TradingBot() as bot:
    bot.run_simulation()
# Cleanup SEMPRE eseguito
```

### 4️⃣ **LRU Cache** - Performance Boost Automatico

```python
@lru_cache(maxsize=128)
def calculate_indicators(self, symbol: str) -> Dict:
    # Calcolo pesante
    return {'rsi': ..., 'macd': ..., 'volume': ...}
```

**Come funziona:**
```python
# Prima chiamata: calcola
bot.calculate_indicators("BTC/USDT")  # 100ms

# Seconda chiamata: cache hit!
bot.calculate_indicators("BTC/USDT")  # <1ms (128x più veloce!)

# Se chiami con parametri diversi:
bot.calculate_indicators("ETH/USDT")  # 100ms (nuovo calcolo)
```

**Quando usarlo:**
- Funzioni pure (stesso input → stesso output)
- Calcoli costosi
- Chiamate API ripetute

### 5️⃣ **Database Context Manager** - Transazioni Sicure

```python
@contextmanager
def _get_db(self):
    conn = sqlite3.connect(self.db_path)
    try:
        yield conn
        conn.commit()  # ✅ Tutto ok
    except Exception as e:
        conn.rollback()  # ❌ Errore → rollback
        raise
    finally:
        conn.close()  # 🔒 Sempre chiuso
```

**Garantisce:**
1. **Atomicità**: O tutto succede, o niente
2. **Consistency**: Dati sempre validi
3. **Cleanup**: Connessione sempre chiusa

# 🎮 PROGETTO 2: Text Adventure Game - Analisi Approfondita

## 🔍 Panoramica del Problema

**Obiettivo**: Creare un gioco testuale interattivo con:
1. Sistema di stanze interconnesse (grafo)
2. Inventario del giocatore con limiti di peso
3. Sistema di comandi flessibile
4. Condizioni di vittoria

## 🧠 Concetti Chiave Spiegati

### 1️⃣ **Enum per Direzioni** - Addio Stringhe Magiche!

```python
class Direction(Enum):
    NORTH = "north"
    SOUTH = "south"
    EAST = "east"
    WEST = "west"
    
    @classmethod
    def from_string(cls, s: str) -> Optional['Direction']:
        for direction in cls:
            if direction.value.startswith(s):
                return direction
        return None
```

**Prima (❌ BAD):**
```python
# Stringhe ovunque - errori runtime!
exits = {"north": "hall", "sout": "entrance"}  # Typo!
```

**Dopo (✅ GOOD):**
```python
# Type-safe - errori compile-time!
exits = {Direction.NORTH: "hall", Direction.SOUTH: "entrance"}
# Se scrivi male, IDE ti avvisa subito!
```

**Vantaggi:**
- Autocomplete nell'IDE
- Refactoring sicuro
- Impossibile fare typo

### 2️⃣ **Dataclass per Entità di Gioco** - Modellazione Dati

```python
@dataclass
class Room:
    name: str
    description: str
    exits: Dict[Direction, str] = field(default_factory=dict)
    items: List[Item] = field(default_factory=list)
    visited: bool = False
```

**⚠️ ATTENZIONE: Perché `field(default_factory=dict)`?**

```python
# ❌ ERRORE COMUNE:
@dataclass
class Room:
    exits: Dict = {}  # ← MALE! Tutti condividono lo stesso dict!

# ✅ CORRETTO:
@dataclass
class Room:
    exits: Dict = field(default_factory=dict)  # ← Ogni istanza ha il suo!
```

**Dimostrazione del bug:**
```python
# Con exits: Dict = {}
room1 = Room("Hall", "...")
room2 = Room("Library", "...")
room1.exits[Direction.NORTH] = "somewhere"
print(room2.exits)  # {'north': 'somewhere'} ← WTF!?

# Con field(default_factory=dict)
room1 = Room("Hall", "...")
room2 = Room("Library", "...")
room1.exits[Direction.NORTH] = "somewhere"
print(room2.exits)  # {} ← Corretto!
```

### 3️⃣ **Game Loop Pattern** - Il Cuore di Ogni Gioco

```python
while not game_over:
    # 1. RENDER - Mostra lo stato attuale
    print(room.get_description())
    
    # 2. INPUT - Ricevi azione dal giocatore
    command = input("> ")
    
    # 3. PROCESS - Elabora il comando
    process_command(command)
    
    # 4. UPDATE - Aggiorna lo stato
    self.moves += 1
    
    # 5. CHECK - Verifica condizioni win/lose
    if check_win():
        game_won()
```

**Questo pattern è usato in:**
- Videogiochi (Unity, Unreal)
- App interattive
- Simulazioni
- Server real-time

### 4️⃣ **Command Pattern** - Parsing Flessibile

```python
def process_command(self, command: str):
    parts = command.split()
    action = parts[0]
    
    if action in ['go', 'move', 'walk'] and len(parts) > 1:
        self.move_player(parts[1])
    elif action in ['n', 'north', 's', 'south']:
        self.move_player(action)  # Shortcut!
    elif action in ['take', 'get'] and len(parts) > 1:
        self.take_item(' '.join(parts[1:]))  # "take golden sword"
```

**Flessibilità:**
- `go north` ✅
- `n` ✅ (shortcut)
- `take map` ✅
- `take golden crown` ✅ (multi-word)

### 5️⃣ **Graph Structure** - Stanze Interconnesse

```
Struttura del castello:
         [Throne Room]
                |
    [Armory] - [Hall] - [Library]
                |
           [Entrance]
```

**Implementazione:**
```python
# Entrance
rooms['entrance'] = Room(
    "Castle Entrance",
    "...",
    exits={Direction.NORTH: 'hall'}  # Connessione
)

# Hall (hub centrale)
rooms['hall'] = Room(
    "Great Hall",
    "...",
    exits={
        Direction.SOUTH: 'entrance',
        Direction.EAST: 'library',
        Direction.WEST: 'armory',
        Direction.NORTH: 'throne'
    }
)
```

**È un grafo!** Ogni stanza è un nodo, ogni uscita è un arco.

# 🤖 PROGETTO 3: Task Automation Tool - Analisi Approfondita

## 🔍 Panoramica del Problema

**Obiettivo**: Creare tool di automazione per:
1. Organizzare file automaticamente per tipo
2. Estrarre informazioni da testo (email, URL, telefoni)
3. Gestire backup con timestamp
4. Verificare integrità file con hashing

## 🧠 Concetti Chiave Spiegati

### 1️⃣ **Pathlib** - File Handling Moderno

```python
from pathlib import Path

# ❌ OLD WAY (evitare):
import os
path = os.path.join('folder', 'subfolder', 'file.txt')
if os.path.exists(path):
    with open(path, 'r') as f:
        content = f.read()

# ✅ NEW WAY (preferire):
path = Path('folder') / 'subfolder' / 'file.txt'
if path.exists():
    content = path.read_text()
```

**Vantaggi Pathlib:**
- Cross-platform (funziona su Windows, Linux, Mac)
- Object-oriented (metodi invece di funzioni)
- Più leggibile
- Meno errori

**Operazioni comuni:**
```python
path = Path('myfile.txt')

# Check
path.exists()      # File esiste?
path.is_file()     # È un file?
path.is_dir()      # È una directory?

# Read/Write
content = path.read_text()
path.write_text("hello")
bytes = path.read_bytes()

# Info
path.name          # "myfile.txt"
path.stem          # "myfile"
path.suffix        # ".txt"
path.parent        # Path to parent dir

# Iterate
for file in path.parent.iterdir():
    if file.is_file():
        print(file.name)
```

### 2️⃣ **Regular Expressions (Regex)** - Pattern Matching Potente

```python
# Pattern per email
pattern = r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b'
emails = re.findall(pattern, text)
```

**Anatomia di un Pattern:**

```
\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b
│  │                │ │               │  │          │
│  │                │ │               │  │          └─ Word boundary
│  │                │ │               │  └─ TLD (2+ chars)
│  │                │ │               └─ Literal dot
│  │                │ └─ Domain
│  │                └─ Literal @
│  └─ Username (letters, numbers, special chars)
└─ Word boundary
```

**Simboli comuni:**
```python
.       # Qualsiasi carattere
\\d      # Digit (0-9)
\\w      # Word char (a-z, A-Z, 0-9, _)
\\s      # Whitespace
[abc]   # a OR b OR c
[a-z]   # Range: a to z
+       # 1 o più
*       # 0 o più
?       # 0 o 1 (opzionale)
{n,m}   # Da n a m ripetizioni
^       # Inizio stringa
$       # Fine stringa
\\b      # Word boundary
()      # Gruppo
(?:)    # Non-capturing group
```

**Esempi pratici:**
```python
# Email
r'[\\w.+-]+@[\\w.-]+\\.[a-z]{2,}'

# URL
r'https?://[^\\s]+'

# Telefono (formato US)
r'\\(\\d{3}\\)\\s?\\d{3}-\\d{4}'  # (123) 456-7890

# Data (DD/MM/YYYY)
r'\\d{2}/\\d{2}/\\d{4}'

# IP Address
r'\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}'
```

### 3️⃣ **File Hashing** - Integrità e Deduplicazione

```python
import hashlib

def calculate_hash(file_path: Path) -> str:
    sha256 = hashlib.sha256()
    sha256.update(file_path.read_bytes())
    return sha256.hexdigest()
```

**Usi pratici:**

**1. Verificare integrità:**
```python
hash_before = calculate_hash(file)
# ... trasferisci file ...
hash_after = calculate_hash(file)
if hash_before == hash_after:
    print("✅ File integro!")
```

**2. Trovare duplicati:**
```python
hash_map = {}
for file in directory.iterdir():
    h = calculate_hash(file)
    if h in hash_map:
        print(f"Duplicate: {file} == {hash_map[h]}")
    hash_map[h] = file
```

**3. Checksum download:**
```python
# Server fornisce: file.zip + file.zip.sha256
downloaded_hash = calculate_hash('file.zip')
expected_hash = Path('file.zip.sha256').read_text()
assert downloaded_hash == expected_hash
```

### 4️⃣ **@classmethod vs @staticmethod** - Quando Usare Cosa?

```python
class FileOrganizer:
    FILE_CATEGORIES = {...}  # Class variable
    
    @classmethod
    def organize_directory(cls, directory):
        # Ha accesso a cls (la classe)
        category = cls._get_category(file)  # Può chiamare altri classmethod
        
    @classmethod
    def _get_category(cls, file_path):
        # Usa class variable
        for category, extensions in cls.FILE_CATEGORIES.items():
            ...
    
    @staticmethod
    def extract_emails(text):
        # Non serve né self né cls
        # È solo una funzione utility legata logicamente alla classe
        return re.findall(pattern, text)
```

**Quando usare:**

**`@staticmethod`:**
- Funzione utility
- Non serve accesso a istanza o classe
- Potrebbe essere funzione globale, ma è logicamente parte della classe

**`@classmethod`:**
- Factory methods
- Metodi alternativi di costruzione
- Serve accesso a class variables
- Ereditarietà (cls cambia nelle sottoclassi)

**Esempio factory method:**
```python
@dataclass
class Person:
    name: str
    age: int
    
    @classmethod
    def from_birth_year(cls, name: str, birth_year: int):
        age = datetime.now().year - birth_year
        return cls(name, age)  # Crea istanza

# Uso
p1 = Person("Alice", 30)  # Costruttore normale
p2 = Person.from_birth_year("Bob", 1990)  # Factory method
```

In [ ]:
## Esercizio 5: Context Manager - Timer

Crea un timer context manager:

```python
import time

class Timer:
    \"\"\"Context manager per misurare tempo di esecuzione\"\"\"
    
    def __enter__(self):
        \"\"\"TODO: Salva tempo di inizio\"\"\"
        # self.start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        \"\"\"TODO: Calcola e stampa tempo trascorso\"\"\"
        # elapsed = time.time() - self.start
        # print(f"Elapsed: {elapsed:.2f} seconds")
        return False

# Test
# with Timer():
#     time.sleep(1)
#     print("Doing work...")
# Output atteso: "Elapsed: 1.00 seconds"
```

# 🎓 RIEPILOGO E PROSSIMI PASSI

## ✅ Cosa Hai Imparato Oggi

### **Concetti Avanzati Python:**
- ✓ **Protocol Pattern** - Interfacce type-safe
- ✓ **Dataclasses** - Meno boilerplate, più validazione
- ✓ **Context Managers** - Gestione automatica risorse
- ✓ **Decorators** - `@lru_cache`, `@contextmanager`, `@classmethod`, `@staticmethod`
- ✓ **Enum** - Costanti type-safe
- ✓ **Type Hints** - Codice più sicuro e documentato

### **Database & I/O:**
- ✓ **SQLite** - Database embedded
- ✓ **Transactions** - Commit/Rollback automatico
- ✓ **Pathlib** - File handling moderno
- ✓ **Context managers per DB** - Connessioni sicure

### **Pattern di Programmazione:**
- ✓ **Strategy Pattern** - Algoritmi intercambiabili
- ✓ **Game Loop** - Render → Input → Process → Update → Check
- ✓ **Command Pattern** - Parsing comandi flessibile
- ✓ **Factory Method** - Costruttori alternativi

### **Text Processing & Automation:**
- ✓ **Regular Expressions** - Pattern matching potente
- ✓ **File Organization** - Automazione per tipo
- ✓ **Hashing** - Integrità e deduplicazione
- ✓ **Backup Management** - Timestamp e versioning

### **Best Practices:**
- ✓ **Logging** - Debug professionale
- ✓ **Error Handling** - Try/except/finally
- ✓ **Type Safety** - Protocol + Type hints
- ✓ **Separation of Concerns** - Single Responsibility

---

## 🚀 Come Procedere

### 1. **Esegui il Codice**
Esegui tutte le celle del notebook dall'inizio alla fine:
- Osserva l'output del Trading Bot
- Studia come funziona ogni componente
- Sperimenta modificando i parametri

### 2. **Completa gli Esercizi**
Risolvi i 5 esercizi proposti:
1. Momentum Strategy
2. Sistema di combattimento
3. Estrazione date con regex
4. File deduplication
5. Timer context manager

### 3. **Estendi i Progetti**
Idee per migliorare:

**Trading Bot:**
- Aggiungi più strategie (MACD, Bollinger Bands)
- Implementa backtesting
- Crea visualizzazioni con matplotlib
- Integra API reali (Binance, Coinbase)

**Adventure Game:**
- Aggiungi sistema di combattimento
- Crea più stanze e puzzle
- Implementa save/load game
- Aggiungi NPC e dialoghi

**Automation Tool:**
- Scheduler per backup automatici
- Notifiche email quando trova duplicati
- GUI con tkinter/PyQt
- Integrazione cloud (Google Drive, Dropbox)

### 4. **Approfondisci i Concetti**
Studia:
- **Design Patterns**: [Refactoring Guru](https://refactoring.guru/design-patterns/python)
- **Type Hints**: [PEP 484](https://peps.python.org/pep-0484/)
- **Async Python**: asyncio, aiohttp
- **Testing**: pytest, unittest, mocking

---

## 📚 Risorse Consigliate

### **Documentazione Ufficiale:**
- [Python Docs](https://docs.python.org/3/)
- [Type Hints Cheat Sheet](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)
- [Pathlib Guide](https://docs.python.org/3/library/pathlib.html)

### **Tutorial & Guide:**
- [Real Python](https://realpython.com/) - Tutorial approfonditi
- [Python Design Patterns](https://refactoring.guru/design-patterns/python)
- [Regex101](https://regex101.com/) - Testa regex interattivamente

### **Tools Utili:**
- **mypy** - Type checker statico
- **black** - Code formatter
- **pylint** - Linter
- **pytest** - Testing framework

---

## 💡 Tips per l'Apprendimento

1. **Scrivi Codice Ogni Giorno** - Anche solo 30 minuti
2. **Leggi Codice Altrui** - Impara da progetti open source
3. **Fai Progetti Personali** - Risolvi problemi reali
4. **Partecipa a Code Review** - Feedback è prezioso
5. **Insegna ad Altri** - Il modo migliore per imparare

---

## 🎯 Challenge Finale

Combina i 3 progetti in uno solo!

**Idea: Trading Adventure Game**
- Gioco testuale dove sei un trader
- Ogni stanza è un mercato diverso (Crypto, Stocks, Forex)
- Devi fare trade per guadagnare oro
- Obiettivo: Raggiungere 1 milione di oro
- Salva progressi in database
- Genera report automatici

**Buon coding! 🚀**

In [ ]:
## Esercizio 4: File Deduplication

Trova file duplicati usando hash:

```python
from pathlib import Path
from typing import Dict, List
import hashlib

def find_duplicates(directory: Path) -> Dict[str, List[Path]]:
    \"\"\"
    TODO: Trova file duplicati per contenuto (hash uguale).
    
    Return: Dict con hash → lista di file con quel hash
    Solo hash con 2+ file (duplicati)
    \"\"\"
    hash_map: Dict[str, List[Path]] = {}
    
    # TODO: Itera sui file in directory
    # TODO: Calcola hash di ogni file
    # TODO: Aggiungi a hash_map[hash]
    # TODO: Filtra solo duplicati
    
    return {}

# Test
# test_dir = Path("test_dup")
# test_dir.mkdir(exist_ok=True)
# (test_dir / "file1.txt").write_text("same content")
# (test_dir / "file2.txt").write_text("same content")
# (test_dir / "file3.txt").write_text("different")

# duplicates = find_duplicates(test_dir)
# print(f"Duplicati: {len(duplicates)} gruppi")
```

In [ ]:
## Esercizio 3: Regex - Estrai Date

Implementa estrazione date da testo:

```python
import re
from typing import List

def extract_dates(text: str) -> List[str]:
    \"\"\"
    TODO: Estrai date nei formati:
    - DD/MM/YYYY (es. 15/03/2024)
    - YYYY-MM-DD (es. 2024-03-15)
    - DD-MM-YYYY (es. 15-03-2024)
    \"\"\"
    patterns = [
        # TODO: Aggiungi pattern regex qui
    ]
    
    results = []
    # TODO: Usa re.findall per ogni pattern
    
    return results

# Test
text = "Meeting on 15/03/2024, deadline 2024-12-31, event 01-01-2025"
dates = extract_dates(text)
print(f"Date trovate: {dates}")
assert len(dates) == 3
```

In [ ]:
## Esercizio 2: Adventure Game - Aggiungi Nemici

Estendi il gioco con sistema di combattimento:

```python
from dataclasses import dataclass

@dataclass
class Enemy:
    name: str
    health: int
    attack: int
    
    def take_damage(self, damage: int):
        \"\"\"TODO: Riduci health, ma non sotto 0\"\"\"
        pass
    
    def is_alive(self) -> bool:
        \"\"\"TODO: Return True se health > 0\"\"\"
        pass

# Test
enemy = Enemy("Dragon", 100, 20)
enemy.take_damage(30)
assert enemy.health == 70
assert enemy.is_alive() == True

enemy.take_damage(100)
assert enemy.health == 0
assert enemy.is_alive() == False
```

In [ ]:
# 💪 ESERCIZI PRATICI

## Esercizio 1: Estendi il Trading Bot

Implementa una strategia Momentum:

```python
class MomentumStrategy:
    \"\"\"Strategia basata su variazione 24h\"\"\"
    
    def analyze(self, data: Dict) -> float:
        \"\"\"
        TODO: Implementa la logica:
        - Se change_24h > 3%: return 1.0 (STRONG BUY)
        - Se change_24h < -3%: return -1.0 (STRONG SELL)
        - Altrimenti: return 0.0 (NEUTRAL)
        \"\"\"
        # TUO CODICE QUI
        pass

# Test
strategy = MomentumStrategy()
assert strategy.analyze({'change_24h': 5.0}) == 1.0
assert strategy.analyze({'change_24h': -4.0}) == -1.0
assert strategy.analyze({'change_24h': 1.0}) == 0.0
```

# 💡 PROBLEM SOLVING: Framework Completo

## 🎯 Metodologia in 7 Step

### STEP 1: Comprendi il Problema
```
❓ Fai queste domande:
1. Qual è l'obiettivo finale?
2. Quali sono input e output?
3. Quali vincoli/limitazioni ho?
4. Ci sono casi edge pericolosi?
5. Quali performance sono richieste?
```

### STEP 2: Scomponi in Sottoproblemi
```
📦 Divid-et-Impera:
Problema Grande
  ├─ Sottoproblema 1
  ├─ Sottoproblema 2
  └─ Sottoproblema 3
```

### STEP 3: Scegli Strutture Dati
```
| Caso d'Uso | Struttura | Complessità |
|------------|-----------|-------------|
| Lookup veloce | Dict | O(1) |
| Ordine importante | List | O(n) |
| Unicità | Set | O(1) |
| FIFO | collections.deque | O(1) |
| Priority | heapq | O(log n) |
```

### STEP 4: Identifica Pattern
```
Pattern comuni visti:
✓ Strategy Pattern → TradingBot
✓ Game Loop → AdventureGame
✓ Context Manager → TradingBot.__enter__/__exit__
✓ Factory Method → @classmethod costruttori
✓ Command Pattern → process_command()
```

### STEP 5: Implementa con Type Safety
```python
# ❌ Senza type hints
def process(data):
    result = {}
    for item in data:
        result[item.id] = item.value
    return result

# ✅ Con type hints
def process(data: List[Item]) -> Dict[int, float]:
    result: Dict[int, float] = {}
    for item in data:
        result[item.id] = item.value
    return result
```

### STEP 6: Error Handling Robusto
```python
# ❌ Pericoloso
def save_trade(trade):
    conn = get_db()
    conn.execute("INSERT ...", trade)
    conn.commit()
    # Cosa succede se c'è un errore? Memory leak!

# ✅ Sicuro
def save_trade(trade: Trade) -> bool:
    try:
        with self._get_db() as conn:
            conn.execute("INSERT ...", trade.to_tuple())
        return True
    except DatabaseError as e:
        logger.error(f"Failed to save trade: {e}")
        return False
    # Cleanup automatico!
```

### STEP 7: Test e Validazione
```python
# Unit test
def test_trade_validation():
    # Valid trade
    trade = Trade("BTC/USDT", 0.1, 43000)
    assert trade.quantity == 0.1
    
    # Invalid quantity
    with pytest.raises(ValueError):
        Trade("BTC/USDT", -0.1, 43000)
    
    # Invalid price
    with pytest.raises(ValueError):
        Trade("BTC/USDT", 0.1, -100)
```

## 📚 Carica il Codice Sorgente

Prima di tutto, carich iamo tutto il codice dal file originale: